In [1]:
import pandas as pd
import duckdb
import numpy as np

In [2]:
con = duckdb.connect(database=':memory:', read_only=False)
con.execute("CREATE TABLE ist AS SELECT * FROM read_csv_auto('IST_corrected.csv')")
raw_size = con.execute("SELECT * FROM ist").fetchdf().size
print(raw_size)

2176720


In [3]:
seperate_df_columns = "AGE, SEX, RSBP, STYPE"

# Strings of comma seperated column names (variables) to be selected from the table, subcaterogized in determined time.
randomisation = "RDEF1, RDEF2, RDEF3, RDEF4, RDEF5, RDEF6, RDEF7, RDEF8, RXASP, RXHEP"
# fourteen_day_form is unused
fourteen_day_form = "DSCH, DIVH, DAP, DOAC, DGORM, DSTER, DCAA, DHAEMD, DCAREND, DTHROMB, DMAJNCH"

final_diagnosis = "DDIAGISC, DDIAGHA, DDIAGUN, DNOSTRK"
recurrent_stroke = "DRSISC, DRSH"
fourteen_day_death = "DPE, DDEAD"
six_months = "OCCODE"
# Join column String's
columns = ', '.join([randomisation, final_diagnosis, recurrent_stroke, fourteen_day_death, seperate_df_columns, six_months])
# Construct dataframe
df = con.execute("SELECT {} FROM ist".format(columns)).fetchdf()
# df_seperate = con.execute("SELECT {0}, {1} FROM ist".format(seperate_df_columns, six_months)).fetchdf()
# Keep track of pre-pre-processed data size
var_list = columns.split(", ")
print(var_list)
initial_size = df.size
print(initial_size)

['RDEF1', 'RDEF2', 'RDEF3', 'RDEF4', 'RDEF5', 'RDEF6', 'RDEF7', 'RDEF8', 'RXASP', 'RXHEP', 'DDIAGISC', 'DDIAGHA', 'DDIAGUN', 'DNOSTRK', 'DRSISC', 'DRSH', 'DPE', 'DDEAD', 'AGE', 'SEX', 'RSBP', 'STYPE', 'OCCODE']
447005


In [4]:
dtypes_before = df.dtypes
print(dtypes_before)

rdef1       object
rdef2       object
rdef3       object
rdef4       object
rdef5       object
rdef6       object
rdef7       object
rdef8       object
rxasp       object
rxhep       object
ddiagisc    object
ddiagha     object
ddiagun     object
dnostrk     object
drsisc      object
drsh        object
dpe         object
ddead       object
age          int32
sex         object
rsbp         int32
stype       object
occode       int32
dtype: object


In [5]:
df.info

<bound method DataFrame.info of       rdef1 rdef2 rdef3 rdef4 rdef5 rdef6 rdef7 rdef8 rxasp rxhep  ...  \
0         N     N     N     Y     N     Y     N     Y     Y     N  ...   
1         Y     Y     Y     N     N     N     N     N     N     L  ...   
2         Y     Y     Y     N     N     N     N     N     Y     N  ...   
3         N     N     N     Y     N     N     N     N     N     H  ...   
4         Y     Y     Y     N     N     N     N     N     Y     H  ...   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
19430     Y     Y     Y     Y     Y     C     N     N     N     L  ...   
19431     Y     Y     Y     N     N     N     N     C     N     M  ...   
19432     Y     Y     Y     N     N     N     N     N     N     N  ...   
19433     N     Y     Y     Y     C     C     N     N     N     N  ...   
19434     Y     Y     Y     N     N     N     Y     N     Y     M  ...   

      dnostrk drsisc drsh dpe ddead age sex rsbp  stype occode  
0           N 

In [6]:
def pre_process_columns(some_df, column):
    if str(column) == 'occode':
        some_df[column] = some_df[column].replace({0 : np.nan, 9 : np.nan})
        some_df[column] = some_df[column].replace({np.nan : np.nan, 2 : 0, 3 : 0, 4 : 0})
    if str(column) == 'sex':
        some_df[column] = some_df[column].replace({'M' : 0, 'F' : 1})
    if str(column) in ['age', 'rsbp', 'occode', 'sex']:
        return
    # All character columns to uppercase
    some_df[column] = some_df[column].apply(lambda x : str(x).upper() if type(x) == object else x)
    if str(column) in ['stype']:
        return
    some_df[column] = some_df[column].replace({'Y': 1, 'M': 1, 'H': 1, 'L': 1, 'N': 0, 'C' : np.nan, 'U' : np.nan})

In [7]:
for col in df.columns:
    pre_process_columns(df, col)

In [8]:
df.info

<bound method DataFrame.info of        rdef1  rdef2  rdef3  rdef4  rdef5  rdef6  rdef7  rdef8  rxasp  rxhep  \
0        0.0    0.0    0.0    1.0    0.0    1.0    0.0    1.0      1      0   
1        1.0    1.0    1.0    0.0    0.0    0.0    0.0    0.0      0      1   
2        1.0    1.0    1.0    0.0    0.0    0.0    0.0    0.0      1      0   
3        0.0    0.0    0.0    1.0    0.0    0.0    0.0    0.0      0      1   
4        1.0    1.0    1.0    0.0    0.0    0.0    0.0    0.0      1      1   
...      ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
19430    1.0    1.0    1.0    1.0    1.0    NaN    0.0    0.0      0      1   
19431    1.0    1.0    1.0    0.0    0.0    0.0    0.0    NaN      0      1   
19432    1.0    1.0    1.0    0.0    0.0    0.0    0.0    0.0      0      0   
19433    0.0    1.0    1.0    1.0    NaN    NaN    0.0    0.0      0      0   
19434    1.0    1.0    1.0    0.0    0.0    0.0    1.0    0.0      1      1   

       ... dnostrk 

In [9]:
df.columns

Index(['rdef1', 'rdef2', 'rdef3', 'rdef4', 'rdef5', 'rdef6', 'rdef7', 'rdef8',
       'rxasp', 'rxhep', 'ddiagisc', 'ddiagha', 'ddiagun', 'dnostrk', 'drsisc',
       'drsh', 'dpe', 'ddead', 'age', 'sex', 'rsbp', 'stype', 'occode'],
      dtype='object')

In [10]:
df.dtypes

rdef1       float64
rdef2       float64
rdef3       float64
rdef4       float64
rdef5       float64
rdef6       float64
rdef7       float64
rdef8       float64
rxasp         int64
rxhep         int64
ddiagisc     object
ddiagha     float64
ddiagun     float64
dnostrk     float64
drsisc      float64
drsh        float64
dpe         float64
ddead       float64
age           int32
sex           int64
rsbp          int32
stype        object
occode      float64
dtype: object

In [11]:
for column in df.columns:
    print("Column {} has {} NaN values.".format(column, df[column].isna().sum()))

Column rdef1 has 247 NaN values.
Column rdef2 has 123 NaN values.
Column rdef3 has 255 NaN values.
Column rdef4 has 584 NaN values.
Column rdef5 has 3945 NaN values.
Column rdef6 has 3448 NaN values.
Column rdef7 has 1592 NaN values.
Column rdef8 has 1249 NaN values.
Column rxasp has 0 NaN values.
Column rxhep has 0 NaN values.
Column ddiagisc has 39 NaN values.
Column ddiagha has 32 NaN values.
Column ddiagun has 29 NaN values.
Column dnostrk has 32 NaN values.
Column drsisc has 27 NaN values.
Column drsh has 24 NaN values.
Column dpe has 23 NaN values.
Column ddead has 25 NaN values.
Column age has 0 NaN values.
Column sex has 0 NaN values.
Column rsbp has 0 NaN values.
Column stype has 0 NaN values.
Column occode has 150 NaN values.


In [12]:
print(df.size)
print(df.shape)
# Drop all rows with any NaN value.
df = df.dropna(axis = 0, how = 'any')
print("### DROPPED NaN ###")
print(df.size)
print(df.shape)

447005
(19435, 23)
### DROPPED NaN ###
315859
(13733, 23)


In [13]:
# Y -> 1
# N -> 0
# Set all types with (1, 0) to int
replace_vars = ', '.join([randomisation, final_diagnosis, recurrent_stroke, fourteen_day_death, six_months]).split(', ')
for col in replace_vars:
    df[col.lower()] = df[col.lower()].astype(int, copy=False)

<ipython-input-13-34cb0d4615c5>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col.lower()] = df[col.lower()].astype(int, copy=False)


In [14]:
df.dtypes

rdef1        int32
rdef2        int32
rdef3        int32
rdef4        int32
rdef5        int32
rdef6        int32
rdef7        int32
rdef8        int32
rxasp        int32
rxhep        int32
ddiagisc     int32
ddiagha      int32
ddiagun      int32
dnostrk      int32
drsisc       int32
drsh         int32
dpe          int32
ddead        int32
age          int32
sex          int64
rsbp         int32
stype       object
occode       int32
dtype: object

In [15]:
df.info

<bound method DataFrame.info of        rdef1  rdef2  rdef3  rdef4  rdef5  rdef6  rdef7  rdef8  rxasp  rxhep  \
0          0      0      0      1      0      1      0      1      1      0   
1          1      1      1      0      0      0      0      0      0      1   
2          1      1      1      0      0      0      0      0      1      0   
3          0      0      0      1      0      0      0      0      0      1   
4          1      1      1      0      0      0      0      0      1      1   
...      ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
19423      1      1      1      1      0      0      0      0      0      0   
19426      1      1      1      1      0      0      0      0      1      1   
19429      0      1      1      0      0      0      0      0      1      1   
19432      1      1      1      0      0      0      0      0      0      0   
19434      1      1      1      0      0      0      1      0      1      1   

       ...  dnostrk

In [16]:
df = pd.concat([df, pd.get_dummies(df.stype)], axis=1)
df.info

<bound method DataFrame.info of        rdef1  rdef2  rdef3  rdef4  rdef5  rdef6  rdef7  rdef8  rxasp  rxhep  \
0          0      0      0      1      0      1      0      1      1      0   
1          1      1      1      0      0      0      0      0      0      1   
2          1      1      1      0      0      0      0      0      1      0   
3          0      0      0      1      0      0      0      0      0      1   
4          1      1      1      0      0      0      0      0      1      1   
...      ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
19423      1      1      1      1      0      0      0      0      0      0   
19426      1      1      1      1      0      0      0      0      1      1   
19429      0      1      1      0      0      0      0      0      1      1   
19432      1      1      1      0      0      0      0      0      0      0   
19434      1      1      1      0      0      0      1      0      1      1   

       ...  age  se

In [17]:
del df['stype']
df.info

<bound method DataFrame.info of        rdef1  rdef2  rdef3  rdef4  rdef5  rdef6  rdef7  rdef8  rxasp  rxhep  \
0          0      0      0      1      0      1      0      1      1      0   
1          1      1      1      0      0      0      0      0      0      1   
2          1      1      1      0      0      0      0      0      1      0   
3          0      0      0      1      0      0      0      0      0      1   
4          1      1      1      0      0      0      0      0      1      1   
...      ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
19423      1      1      1      1      0      0      0      0      0      0   
19426      1      1      1      1      0      0      0      0      1      1   
19429      0      1      1      0      0      0      0      0      1      1   
19432      1      1      1      0      0      0      0      0      0      0   
19434      1      1      1      0      0      0      1      0      1      1   

       ...  ddead  

In [18]:
df.to_csv('IST_corrected_processed.csv',index=False)

In [19]:
df['age'].mean()

70.44607878832011

In [20]:
df['occode'].mean()

0.14869292944003495

In [21]:
df['ddead'].mean()

0.061530619675234835

In [22]:
df['rsbp'].mean()

160.467487074929

In [23]:
df['rdef1'].mean()

0.698026651132309